In [60]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet

--2023-05-23 08:01:27--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.51, 65.8.245.178, 65.8.245.50, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38139949 (36M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-01.parquet.2’

yellow_tripdata_202 100%[===================>]  36.37M  79.2MB/s    in 0.5s    

2023-05-23 08:01:28 (79.2 MB/s) - ‘yellow_tripdata_2022-01.parquet.2’ saved [38139949/38139949]



In [61]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

--2023-05-23 08:01:28--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.51, 65.8.245.178, 65.8.245.50, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45616512 (44M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-02.parquet.2’

yellow_tripdata_202 100%[===================>]  43.50M  83.8MB/s    in 0.5s    

2023-05-23 08:01:29 (83.8 MB/s) - ‘yellow_tripdata_2022-02.parquet.2’ saved [45616512/45616512]



In [62]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt  

In [63]:
df_jan = pd.read_parquet("yellow_tripdata_2022-01.parquet")
df_feb = pd.read_parquet("yellow_tripdata_2022-02.parquet")

Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?

In [64]:
df_jan.shape

(2463931, 19)

Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?


In [65]:
#Compute duration variable for the two DFs
df_jan = df_jan.assign(duration=df_jan["tpep_dropoff_datetime"]-df_jan["tpep_pickup_datetime"])
df_feb = df_feb.assign(duration=df_feb["tpep_dropoff_datetime"]-df_feb["tpep_pickup_datetime"])

In [66]:
#çompute trip duration in minutes
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds()/60)
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds()/60)

In [67]:
#Calculate std of duration
df_jan.duration.std()


46.44530513776802

Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [68]:
#Count percentage of duration outliers
outliers_jan = (df_jan[(df_jan["duration"]>=1) & (df_jan["duration"]<=60)]["duration"].count()/df_jan["duration"].count())*100
outliers_jan

98.27547930522405

In [69]:
#Remove outliers
df_jan_no_outliers = df_jan[(df_jan["duration"]>=1) & (df_jan["duration"]<=60)]
df_jan_no_outliers.shape

(2421440, 20)

In [70]:
#Number of records to be removed which are not between 1 and 60mins
df_jan_removed = df_jan[(df_jan.duration<1)|(df_jan.duration >60)]
df_jan_removed.shape

(42491, 20)

Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [71]:
# Subset the DataFrame
subset_data = df_jan[['PULocationID', 'DOLocationID']]

In [72]:
# Convert DataFrame to list of dictionaries
dict_list = subset_data.to_dict(orient='records')

In [73]:
# Fit a dictionary vectorizer
vectorizer = DictVectorizer(sparse=False)
vectorizer.fit(dict_list)

DictVectorizer(sparse=False)

In [74]:
# Get the feature matrix
feature_matrix = vectorizer.transform(dict_list)
feature_matrix.shape[1]


2

In [75]:
# Get the number of columns
num_columns = feature_matrix.shape[1]
num_columns

2

In [76]:
#count of missing values
df_jan.isnull().sum(axis = 0)

VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          71503
trip_distance                0
RatecodeID               71503
store_and_fwd_flag       71503
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     71503
airport_fee              71503
duration                     0
dtype: int64

In [77]:
#calculate the missing values
miss_val_percent = 100 * df_jan.isnull().sum() / len(df_jan)
miss_val_percent

VendorID                 0.000000
tpep_pickup_datetime     0.000000
tpep_dropoff_datetime    0.000000
passenger_count          2.901989
trip_distance            0.000000
RatecodeID               2.901989
store_and_fwd_flag       2.901989
PULocationID             0.000000
DOLocationID             0.000000
payment_type             0.000000
fare_amount              0.000000
extra                    0.000000
mta_tax                  0.000000
tip_amount               0.000000
tolls_amount             0.000000
improvement_surcharge    0.000000
total_amount             0.000000
congestion_surcharge     2.901989
airport_fee              2.901989
duration                 0.000000
dtype: float64

In [78]:
#replace the missing values with -1
df_jan = df_jan.replace(np.nan,-1)

In [79]:
PUmiss_val_percent = 100 * len(df_jan[df_jan.PULocationID == -1.0]) / len(df_jan)
PUmiss_val_percent

0.0

In [80]:
DOmiss_val_percent = 100 * len(df_jan[df_jan.DOLocationID == -1.0]) / len(df_jan)
DOmiss_val_percent

0.0

In [81]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [82]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    #çompute trip duration in minutes
    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    #Select all trips less than 60
    df = df[(df.duration>=1) & (df.duration <=60)]
    
    #fill null values with -1
    df = df.replace(np.nan,-1)

    #Extract the most useful categorical and numerical variables from our dataset
    categorical = ['PULocationID','DOLocationID']

    #Convert numerical variables to categorical variables
    df[categorical] = df[categorical].astype(str)
    
    return df

In [83]:
df_train = read_dataframe("yellow_tripdata_2022-01.parquet")
df_val = read_dataframe("yellow_tripdata_2022-02.parquet")
df_train.shape, df_val.shape

((2421440, 20), (2918187, 20))

In [84]:
#Extract the most useful categorical variables from our dataset
categorical = ['PULocationID','DOLocationID']

#perform data vectorization
dv = DictVectorizer()

In [85]:
#Convert our selected features to dictionaries for our vectorizers to work and fit
train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
X_train.shape

(2421440, 515)

In [86]:
#Convert our selected features to dictionaries for our vectorizers to work
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [87]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [88]:
#Apply Linear regression on Training dataset

#train our model
lr = LinearRegression()
lr.fit(X_train, y_train)

#model prediction
y_pred = lr.predict(X_train)

#evaluate our model
rmse_train = mean_squared_error(y_train, y_pred, squared=False)
rmse_train

6.986190742248472

Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

In [89]:
#Apply Linear regression on Validation dataset

#model prediction
y_pred = lr.predict(X_val)

#evaluate our model
rmse_val = mean_squared_error(y_val, y_pred, squared=False)

rmse_val

7.78640662117552